Сейчас будет много копипаста с тетрадки семинара)

In [ ]:
!pip install nltk

In [ ]:
!pip install spacy

In [ ]:
!pip  install pyLDAvis

In [ ]:
import nltk; nltk.download('stopwords')

!python3 -m spacy download en

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


Чистка даты

In [ ]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [ ]:
#tokenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


программа, которая нам руками будет возвращать N-число самых частых биграмм в тексте

In [ ]:
bigrams = [b for l in data_words for b in zip(l[:-1], l[1:])]
freq = nltk.FreqDist(bigrams) #computes freq of occurrence
fdist = freq.keys() # sorted according to freq

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


Mallen, который смог. Будь как маллет.

In [ ]:
!unzip /content/mallet-2.0.8.zip

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

## Тренеруем нашу модель

Надо посчитать лучшее количество топиков через функцию. Шаг = 5, потому что коллаб слишком тупой, а на моем компе не работает spacy :D


In [ ]:
from tqdm.auto import tqdm

In [ ]:
def opt_topics(n, corpus, id2word, data_lemmatized):
  groups_count = []
  for number in tqdm(n):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=number, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
  
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    groups_count.append((coherence_lda, number))
  
  for pair in groups_count:
    print('The number of groups: ', pair[1], ' - ', 'coherence: ', pair[0], '\n', '--------------------')
  
  return sorted(groups_count)[-1][1]

In [ ]:
n = [5, 10, 15, 20]
best_topics_count = opt_topics(n, corpus, id2word, data_lemmatized)


The number of groups:  5  -  coherence:  0.4297369063851419 
 --------------------
The number of groups:  10  -  coherence:  0.4973497938960755 
 --------------------
The number of groups:  15  -  coherence:  0.44582805103485196 
 --------------------
The number of groups:  20  -  coherence:  0.4392813747423439 
 --------------------


Видно, что 10 топиков показывают лучший результат. Возможно, между 10 и 15 топиками есть и лучше.


In [ ]:
best_topics_count

10

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=best_topics_count, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
topics = lda_model.show_topics(num_topics=best_topics_count, num_words=10, formatted=False) #беру show_topics, потому что с print_topics не получается сделать кортежи
doc_lda = lda_model[corpus] #не знаю зачем эта штука, но она была в семе, хуже не будет)

In [ ]:
topics[0][1][:10]

[('team', 0.02816584),
 ('year', 0.02723164),
 ('game', 0.025724994),
 ('play', 0.019548114),
 ('win', 0.018154599),
 ('player', 0.014175091),
 ('run', 0.009828578),
 ('last', 0.009554995),
 ('good', 0.0093642585),
 ('hit', 0.008524719)]

In [ ]:
def d_sorter(d):
  list_d = list(d.items())
  list_d.sort(key=lambda i: i[1])
  return list_d[-1][0]

In [ ]:
topics_values = []

#for index, text in enumerate(topics)
#list_of_topic[0] - index topic
for index, text in tqdm(enumerate(data_lemmatized)):
  topic_value = {}
  for t_index, topic in topics:

    for word in text: #слово в тексте !!!
      for word_in_topic in topic: #('year', 0.02723164)
        if word in word_in_topic[0]: #если слово попадает в этот топик, то...
          try:
            topic_value[t_index] = topic_value[t_index] + word_in_topic[1]
          except:
            topic_value[t_index] = word_in_topic[1]

  try:
    cur_text_topic = d_sorter(topic_value)
    topics_values.append((index, text, cur_text_topic))
  except:
    topics_values.append((index, text, 'N/A'))

In [145]:
df_text = pd.DataFrame()
df_text['text'] = [text for text in data]
df_text['lemmas'] = [i for i in data_lemmatized]
df_text['general_topic'] = [i[2] for i in topics_values]

In [229]:
df_text

,text,lemmas,general_topic
0,From: (wheres my thing) Subject: WHAT car is t...,"[where, thing, car, nntp_poste, host, park, li...",3
1,From: (Guy Kuo) Subject: SI Clock Poll - Final...,"[poll, final, call, summary, final, call, cloc...",6
2,From: (Thomas E Willis) Subject: PB questions....,"[engineering, computer, network, distribution_...",6
3,From: (Joe Green) Subject: Re: Weitek P9000 ? ...,"[division, line, host, write, write, article, ...",7
4,From: (Jonathan McDowell) Subject: Re: Shuttle...,"[question, distribution, article, write, clear...",7
...,...,...,...
11309,From: (Jim Zisfein) Subject: Re: Migraines and...,"[scan, city, reply, line, consultation, cheap,...",7
11310,From: Subject: Screen Death: Mac Plus/512 Line...,"[screen, medford, old, problem, screen, blank,...",1
11311,From: (Will Estes) Subject: Mounting CPU Coole...,"[este, mount, mail, group, line, instal, try, ...",6
11312,From: (Steven Collins) Subject: Re: Sphere fro...,"[line, article, write, boy, embarasse, trivial...",7


TF

In [209]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [298]:
for topic in df_text.general_topic.unique():
  documents = []
  n=0
  for text in df_text['text']:
    if df_text['general_topic'][n] == topic:
      documents.append(text)
    n+=1
  
  vectorizer = TfidfVectorizer()
  vectors = vectorizer.fit_transform(documents)
  feature_names = vectorizer.get_feature_names()
  dense = vectors.todense()
  denselist = dense.tolist()
  df = pd.DataFrame(denselist, columns=feature_names) 



  break

In [294]:
y=sorted(df.loc[0], reverse=True)
y[:5]

[0.20805172056878865,
 0.2053408372500451,
 0.20049226549325574,
 0.1712960027830327,
 0.15789569709271017]

In [332]:
y = df.loc[0]
y=y.sort_values().values


AttributeError: ignored

In [297]:
df.columns_to

,00,000,000042,000094,0001x7c,0002,000miles,01,013,01451,015,01730,01801,018801285,02,0200,02115,0226,03,030,032905,04,0410,05,0500,0518,053,0582,06,0600,0608,06482,0667,07,08,0815,0895,09,0_,0a,...,youngstown,your,youre,yourre,yours,yourself,yourselfers,youth,youths,youve,yr,ysr50s,ysu,za,zahal,zaken,zakir,zal,zauberer,zealand,zeeland,zephyr,zeppos,zero,zeus,zeven,zien,zijn,zip,zippppppppppp,zo,zogeheten,zolmer,zone,zullen,zwaartepunten,zwak,zwakke,zware,zwarte
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.055656,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.050872,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.051321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.041551,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
249,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055072,0.0,0.0,0.0,0.0,...,0.0,0.020859,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
250,0.015308,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005979,0.0,0.0,0.0,0.0,0.0,0.0,0.006141,0.008051,0.0,0.0,0.0,0.008051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.005675,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006537,0.0,0.0,0.0,0.0,0.007492,0.000000,0.0,0.0,0.0,0.028383,0.0,0.0,0.0,0.0,0.0,0.0
251,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [138]:
def WordDict(text):
  numOfWords = dict.fromkeys(text, 0)
  for word in text:
    numOfWords[word] += 1
  return numOfWords

In [136]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [ ]:
computeTF(WordDict(data_lemmatized[0]), data_lemmatized[0])

In [207]:
for topic in df_text.general_topic.unique():
  documents = []
  n=0
  for text in df_text['lemmas']:
    if df_text['general_topic'][n] == topic:
      document = WordDict(text)
      documents.append(document)
    n+=1
  computeIDF(documents)
  break

KeyError: ignored

In [204]:
import math
def computeIDF(documents):
    N = len(documents)
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents: # 
      for word, val in document.items():
        if val > 0:
          idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [161]:
for topic in df_text.general_topic.unique():
  documents = []
  n=0
  for text in df_text['lemmas']:
    if df_text['general_topic'][n] == topic:
      documents.append(text)
  print(documents[:10])
  computeIDF(documents)
  break

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['poll', 'final', 'call', 'summary', 'final', 'call', 'clock', 'report', 'keyword', 'acceleration', 'line', 'host', 'fair', 'number', 'brave', 'soul', 'upgrade', 'clock', 'oscillator', 'share', 'experience', 'poll', 'send', 'brief', 'message', 'detailing', 'experience', 'procedure', 'top', 'speed', 'attain', 'cpu', 'rate', 'speed', 'add', 'card', 'hour', 'usage', 'day', 'functionality', 'floppy', 'especially', 'request', 'summarize', 'next', 'day', 'add', 'network', 'knowledge', 'base', 'do', 'clock', 'upgrade', 'answer', 'poll', 'thank'], ['engineering', 'compu

AttributeError: ignored

In [157]:
  n=0
  for text in df_text['lemmas']:
    print(df_text['general_topic'][n])
    n+=1

Выходные данные были обрезаны до нескольких последних строк (5000).
7
7
6
7
6
7
0
7
7
6
7
7
7
7
0
3
7
6
7
6
0
6
7
7
7
7
7
7
6
7
7
6
7
6
7
7
7
7
7
7
6
7
6
7
6
7
6
6
7
6
7
6
6
0
7
6
7
7
7
7
7
7
7
7
6
7
7
1
0
6
6
3
7
6
7
7
7
0
6
6
7
7
6
7
6
7
7
9
7
6
6
7
7
6
6
7
7
7
6
6
6
1
7
3
7
3
7
6
7
6
7
7
7
7
6
7
6
7
7
6
7
7
1
7
7
6
7
7
7
0
0
7
1
7
6
7
8
7
6
7
7
7
7
7
1
7
7
7
6
7
6
6
5
7
6
7
7
7
7
6
7
6
6
7
6
6
6
7
6
6
7
7
7
0
7
6
7
7
6
7
6
7
6
6
6
7
7
9
7
6
3
6
7
7
7
6
6
7
6
7
6
7
6
6
7
7
7
6
7
6
7
7
5
8
6
6
7
7
6
7
7
0
7
6
7
7
7
7
7
7
6
6
6
7
7
0
7
7
6
9
7
7
6
9
7
9
6
7
7
7
7
1
7
7
7
7
6
7
9
7
6
7
0
6
7
3
3
6
7
7
7
7
0
7
7
7
3
7
7
7
6
7
7
7
6
6
6
7
7
6
7
6
7
7
7
7
6
7
7
6
7
0
6
6
7
7
7
8
9
6
7
6
7
6
7
7
6
7
7
6
7
7
6
7
7
7
6
7
6
9
7
7
7
0
7
3
7
7
6
6
7
6
7
7
6
7
6
0
8
6
6
6
7
0
0
7
7
1
7
7
7
7
7
6
7
6
7
7
1
7
7
6
6
1
7
7
7
7
6
7
8
6
7
7
3
7
7
7
6
5
7
7
2
7
6
7
7
6
6
7
9
7
7
7
9
7
6
7
7
7
6
7
3
7
6
7
7
7
6
6
7
6
7
7
6
7
7
7
6
7
8
7
6
6
7
7
7
7
7
7
0
6
7
6
6
6
7
6
7
3
7
6
6
7
7
7
7
6
7
0
6
7
1
7
7
7
